# Superface Agent Hub - MistralAI Function Calling Example

In this notebook we demonstrate how to use the Superface Agent Hub to connect your MistralAI powered agent to external tools and APIs in a way that allows for both personal and third-party use.

In [ ]:
%pip install pandas "mistralai>=0.1.2" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import json
import requests as r
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown

# Set a random number of your choice, but don't change it
# once you have run the notebook, otherwise you will create another user.
SUPERFACE_USER_ID_CONSTANT = 

# Use the number to create a unique ID
SUPERFACE_USER_ID = "sfoaidemo|" + str(SUPERFACE_USER_ID_CONSTANT)

# Default URL for Superface
SUPERFACE_BASE_URL = "https://pod.superface.ai/api/hub"

# Set the Superface authentication token
SUPERFACE_AUTH_TOKEN="<your-superface-auth-token>"

# Mistral API Key
MISTRAL_API_KEY = "<your-mistral-api-key>"

messages = []

In [ ]:
# Setup MistralAI
model = "mistral-large-latest"
client = MistralClient(api_key="MISTRAL_API_KEY")

In [ ]:
# Define the helper functions used to get and perform functions

def get_superface_tools():
  headers = {"Authorization": "Bearer "+ SUPERFACE_AUTH_TOKEN}
  tools = r.get(SUPERFACE_BASE_URL + "/fd", headers=headers)
  return tools.json()

# Helper function to perform the action for all the functions.
# This is the only API call required regardless of what the function is.
def perform_action(tool_name=None, tool_body=None):
  headers = {"Authorization": "Bearer "+ SUPERFACE_AUTH_TOKEN, "x-superface-user-id": SUPERFACE_USER_ID}
  perform_result = r.post(SUPERFACE_BASE_URL + "/perform/" + tool_name, headers=headers, json=tool_body)
  return json.dumps(perform_result.json())

In [ ]:
prompt = "What is the weather in Prague?" # @param {type:"string"}

messages = [
    ChatMessage(role="user", content=prompt)
]


In [ ]:
response = client.chat(
    model=model,
    messages=messages,
    tools=get_superface_tools(),
    tool_choice="auto"
)

response

ChatCompletionResponse(id='7253f94922b04f1cbca70aad050868ac', object='chat.completion', created=1711469166, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='weather__current-weather__CurrentWeather', arguments='{"city": "Prague, Czech Republic"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=4201, total_tokens=4234, completion_tokens=33))

In [ ]:
messages.append(response.choices[0].message)

In [ ]:
# Extract tool intents and params from the assistant response
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)

# Pass the function name and arguments to Superface
run_function = perform_action(function_name, function_params)

messages.append(ChatMessage(role="tool", name=function_name, content=run_function))
messages

[ChatMessage(role='user', content='What is the weather in Prague?', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='weather__current-weather__CurrentWeather', arguments='{"city": "Prague, Czech Republic"}'))]),
 ChatMessage(role='tool', content='{"status": "success", "assistant_hint": "Format the result in \'result\' field to the user. If the user asked for a specific format, respect it", "result": {"description": "Sunny", "feelsLike": 13, "temperature": 13}}', name='weather__current-weather__CurrentWeather', tool_calls=None)]

In [ ]:
response = client.chat(
  model=model,
  messages=messages
)

display(Markdown(response.choices[0].message.content))

The current weather in Prague, Czech Republic is sunny with a temperature of 13°C. It feels like 13°C.